In [1]:
import sys

sys.path.append("..")
from cge_modeling import CGEModel, Variable, Parameter, Equation
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import sympy as sp

In [2]:
df = pd.read_csv("data/lesson_5_sam.csv", index_col=[0, 1], header=[0, 1]).map(float).fillna(0)
assert np.allclose(df.sum(axis=0), df.sum(axis=1))
df

Factor         Institution                  \
                                   Labor Capital   Household   Firms    Govt   
Factor      Labor                    0.0     0.0         0.0     0.0     0.0   
            Capital                  0.0     0.0         0.0     0.0     0.0   
Institution Household             6950.0     0.0         0.0  2600.0     0.0   
            Firms                    0.0  2600.0         0.0     0.0     0.0   
            Govt                     0.0     0.0      1400.0     0.0     0.0   
Production  Agriculture              0.0     0.0      1700.0     0.0   400.0   
            Industry                 0.0     0.0      1200.0     0.0   900.0   
            Services                 0.0     0.0      3400.0     0.0  2750.0   
Activities  Agriculture              0.0     0.0         0.0     0.0     0.0   
            Industry                 0.0     0.0         0.0     0.0     0.0   
            Services                 0.0     0.0         0.0     0.0     0.0   
Other       Capital Accumulation     0.0     0.0      1850.0     0.0  -900.0   
            Rest of World            0.0     0.0         0.0     0.0     0.0   

                                  Production                    Activities  \
                                 Agriculture Industry Services Agriculture   
Factor      Labor                        0.0      0.0      0.0      1100.0   
            Capital                      0.0      0.0      0.0       250.0   
Institution Household                    0.0      0.0      0.0         0.0   
            Firms                        0.0      0.0      0.0         0.0   
            Govt                       150.0   1320.0    280.0         0.0   
Production  Agriculture                  0.0      0.0      0.0      1100.0   
            Industry                     0.0      0.0      0.0      2200.0   
            Services                     0.0      0.0      0.0       850.0   
Activities  Agriculture               5500.0      0.0      0.0         0.0   
            Industry                     0.0   9800.0      0.0         0.0   
            Services                     0.0      0.0  10200.0         0.0   
Other       Capital Accumulation         0.0      0.0      0.0         0.0   
            Rest of World                0.0    430.0      0.0         0.0   

                                                                  Other  \
                                 Industry Services Capital Accumulation   
Factor      Labor                  1700.0   4150.0                  0.0   
            Capital                1800.0    550.0                  0.0   
Institution Household                 0.0      0.0                  0.0   
            Firms                     0.0      0.0                  0.0   
            Govt                      0.0      0.0                  0.0   
Production  Agriculture             900.0   1100.0                200.0   
            Industry               3150.0   3300.0                800.0   
            Services               2250.0   1100.0                100.0   
Activities  Agriculture               0.0      0.0                  0.0   
            Industry                  0.0      0.0                  0.0   
            Services                  0.0      0.0                  0.0   
Other       Capital Accumulation      0.0      0.0                  0.0   
            Rest of World             0.0      0.0                  0.0   

                                                
                                 Rest of World  
Factor      Labor                          0.0  
            Capital                        0.0  
Institution Household                      0.0  
            Firms                          0.0  
            Govt                           0.0  
Production  Agriculture                  250.0  
            Industry                       0.0  
            Services                      30.0  
Activities  Agriculture                    0.0

In [53]:
variable_info = [
    # Firm variables (7)
    Variable(name="Y", dims="i", description="Final output in the <dim:i> sector"),
    Variable(name="VA", dims="i", description="Value-added component of <dim:i> sector production"),
    Variable(name="VC", dims="i", description="Value-chain component of <dim:i> sector production"),
    Variable(
        name="X",
        dims=("i", "j"),
        description="Demand for <dim:i> sector goods by the <dim:j> sector as value-chain inputs",
    ),
    Variable(
        name="L_d",
        dims="i",
        extend_subscript=True,
        description="Labor demand in the <dim:i> sector",
    ),
    Variable(
        name="K_d",
        dims="i",
        extend_subscript=True,
        description="Capital demand in the <dim:i> sector",
    ),
    Variable(
        name="I_d",
        dims="i",
        extend_subscript=True,
        description="Investment capital demanded by the <dim:i> sector",
    ),
    # Prices (7)
    Variable(name="P", dims="i", description="Final good price in the <dim:i> sector, after taxes"),
    Variable(
        name="P_VA",
        dims="i",
        extend_subscript=True,
        description="Price of the value-add component in the <dim:i> sector",
    ),
    Variable(
        name="P_VC",
        dims="i",
        extend_subscript=True,
        description="Price of the value-chain component in the <dim:i> sector",
    ),
    Variable(
        name="P_Y",
        dims="i",
        extend_subscript=True,
        description="Final good price in the <dim:i> sector, before taxes",
    ),
    Variable(
        name="P_w", dims="i", extend_subscript=True, description="World price of <dim:i> goods"
    ),
    Variable(name="r", description="Rental rate of capital"),
    Variable(name="w", description="Wage level"),
    # Household Variables (6)
    Variable(name="U", description="Household utility"),
    Variable(name="C", dims="i", description="Household consumption of <dim:i> goods"),
    Variable(
        name="I_s", extend_subscript=True, description="Investment capital supplied by households"
    ),
    Variable(name="S", description="Household savings"),
    Variable(name="income", latex_name="Omega", description="Household income, before taxes"),
    Variable(
        name="net_income", latex_name="\hat{\Omega}", description="Household income, after taxes"
    ),
    # Government variables (3)
    Variable(name="G", description="Government budget"),
    Variable(
        name="C_G",
        dims="i",
        extend_subscript=True,
        description="Government consumption of <dim:i> goods",
    ),
    Variable(
        name="S_G",
        extend_subscript=True,
        positive=None,
        description="Investment capital supplied by government",
    ),
    # International variables (3)
    Variable(
        name="E_d",
        dims="i",
        extend_subscript=True,
        positive=None,
        description="Excess domestic demand for <dim:i> sector goods",
    ),
    Variable(name="e", description="Real exchange rate"),
    Variable(name="TB", description="Trade balance", positive=None),
    # Misc (1)
    Variable(name="resid", latex_name=r"varepsilon", description="Walrasian residual"),
]


param_info = [
    # Production Parameters
    Parameter(
        "alpha",
        dims="i",
        description="Share of capital in production of the <dim:i> sector value-add bundle",
    ),
    Parameter(
        "alpha_k",
        dims="i",
        extend_subscript=True,
        description="Share of capital investment demanded by the <dim:i> sector",
    ),
    Parameter("A", dims="i", description="Total factor productivity of the <dim:i> sector"),
    Parameter(
        "psi_VA",
        extend_subscript=True,
        dims="i",
        description="Share of value-add bundle in <dim:i> sector final good production",
    ),
    Parameter(
        "psi_VC",
        extend_subscript=True,
        dims="i",
        description="Share of value chain bundle in <dim:i> sector final good production",
    ),
    Parameter(
        "psi_X",
        extend_subscript=True,
        dims=("i", "j"),
        description="Share of <dim:j> sector final goods in the <dim:i> value chain bundle",
    ),
    # Tax parameters
    Parameter("tau", dims="i", description="Sales tax rate on final goods in the <dim:i> sector"),
    Parameter(
        "tau_m", dims="i", extend_subscript=True, description="Tariff on <dim:i> sector imports"
    ),
    Parameter("tau_income", latex_name=r"\tau_{\Omega}", description="Income tax rate"),
    # Household parameters
    Parameter(
        "gamma",
        dims="i",
        description="Household elasticity of consumption utility for <dim:i> sector goods",
    ),
    Parameter("mps", latex_name="phi", description="Household marginal propensity to save"),
    # Government parameters
    Parameter(
        "alpha_G",
        dims="i",
        description="Share of <dim:i> sector final goods in governmnet consumption",
    ),
    # Trade concepts
    Parameter(
        "f",
        dims="i",
        description="Scale of effect of domestic <dim:i> sector on world prices.",
        positive=None,
    ),
    Parameter(
        "sigma",
        dims="i",
        description="Elasticity of domestic demand for <dim:i> sector goods on world prices.",
        positive=None,
    ),
    # Exogenous values
    Parameter("L_s", description="Exogenous labor supply"),
    Parameter("K_s", description="Exogenous capital supply"),
    Parameter("P_Ag_bar", latex_name=r"\bar{P}_{Ag}", description="Numeraire price"),
    Parameter("e_bar", latex_name=r"\bar{e}", description="Exogenous exchange rate"),
    Parameter(
        "S_G_bar",
        latex_name=r"\bar{S}_G",
        description="Exogenous level of governmnet savings",
        positive=None,
    ),
]


equations = [
    # Frim Block
    Equation("Final good production of sector <dim:i>", "P_Y * Y = P_VC * VC + P_VA * VA"),
    Equation("Sector <dim:i> demand for value chain bundle", "VC = psi_VC * Y"),
    Equation("Sector <dim:i> demand for value-add bundle", "VA = psi_VA * Y"),
    Equation("Sector <dim:i> production of labor-capital", "VA = A * K_d^alpha * L_d^(1-alpha)"),
    Equation("Sector <dim:i> demand for capital", "K_d = alpha * VA * P_VA / r"),
    Equation("Sector <dim:i> demand for labor", "L_d = (1 - alpha) * VA * P_VA / w"),
    Equation(
        "Sector <dim:i> production of value chain bundle",
        "VC * P_VC = Sum(P.subs({i:j}) * X.subs([(i,k), (j,i), (k,j)]), (j, 0, 2))",
    ),
    Equation("Demand for <dim:i> sector goods by <dim:j>", "X = psi_X * VC.subs({i:j})"),
    Equation("<dim:i> sector demand for investment capital", "P * I_d = alpha_k * I_s"),
    # Government block
    Equation(
        "Government budget constraint",
        "G + S_G = tau_income * income + Sum(tau * P_Y * Y + Max(0, E_d) * tau_m * e * P_w, (i, 0, 2))",
    ),
    Equation("Government consumption of <dim:i> sector goods", "P * C_G = alpha_G * G"),
    Equation("Sector <dim:i> after-tax price", "P = (1 + tau) * P_Y"),
    Equation("Exogenous government savings", "S_G = S_G_bar"),
    # Household block
    Equation("Household pre-tax income", "income = w * L_s + r * K_s"),
    Equation("Household after-tax income", "net_income = (1 - tau_income) * income"),
    Equation("Household utility", "U = Product(C**gamma, (i, 0, 2))"),
    Equation("Household demand for good <dim:i>", "C * P = gamma * (1 - mps) * net_income"),
    Equation("Household savings", "S = mps * net_income"),
    # International concepts
    Equation("Law of one price", "P = e * P_w * (1 + tau_m)"),
    Equation("World price", "E_d = f * P_w ** sigma"),
    Equation("Trade balance", "TB = -e * Sum(P_w * E_d , (i, 0, 2))"),
    Equation("Exogenous exchange rate", "e = e_bar"),
    # Economic equilibrium
    Equation("Capital and current account equlibrium", "S + S_G = I_s + TB + resid"),
    Equation("Labor market clearing", "L_s = Sum(L_d, (i, 0, 2))"),
    Equation("Capital market clearing", "K_s = Sum(K_d, (i, 0, 2))"),
    Equation("<dim:i> sector goods market clearing", "Y + E_d = C + Sum(X, (j, 0, 2)) + C_G + I_d"),
    Equation("Numeraire", 'P.subs({i:"Ag"}) = P_Ag_bar'),
]

assert len(variable_info) == len(
    equations
), f"Found {len(variable_info)} variables but {len(equations)} equations"

In [54]:
sectors = ["Ag", "Ind", "Serv"]
coords = {"i": sectors, "j": sectors}

mod = CGEModel(variables=variable_info, parameters=param_info, equations=equations, coords=coords)

In [66]:
mod.equation_table()

<IPython.core.display.Latex object>

In [72]:
for i, eq in enumerate(mod.unpacked_equation_symbols):
    print(i)
    display(eq)
    display(eq.subs({mod.get_symbol(k): v for k, v in initial_values.items()}))
    print("-" * 100)

0


-P_VA_Ag*VA_Ag - P_VC_Ag*VC_Ag + P_Y_Ag*Y_Ag

-VA_Ag - VC_Ag + 5500.0

----------------------------------------------------------------------------------------------------
1


-P_VA_Ind*VA_Ind - P_VC_Ind*VC_Ind + P_Y_Ind*Y_Ind

...

0

----------------------------------------------------------------------------------------------------
58


P_Ag - P_Ag_bar

1 - P_Ag_bar

----------------------------------------------------------------------------------------------------


In [55]:
short_to_long = {"Ag": "Agriculture", "Ind": "Industry", "Serv": "Services"}

# Normalize prices
initial_values = {"w": 1, "r": 1, "e": 1, "resid": 0}
initial_values.update({f"P_VA_{s}": 1 for s in sectors})
initial_values.update({f"P_VC_{s}": 1 for s in sectors})
initial_values.update({f"P_{s}": 1 for s in sectors})

# Values provided by the exercise (from where?)
initial_values["import_tax_Ag"] = 0
initial_values["import_tax_Ind"] = 300
initial_values["import_tax_Serv"] = 0

initial_values["sigma_Ag"] = -3
initial_values["sigma_Ind"] = 3
initial_values["sigma_Serv"] = -3

# Enter data from SAM
initial_values["income_tax"] = df.loc[("Institution", "Govt"), ("Institution", "Household")]
initial_values["S"] = df.loc[("Other", "Capital Accumulation"), ("Institution", "Household")]
initial_values["S_G"] = df.loc[("Other", "Capital Accumulation"), ("Institution", "Govt")]

# Aggregates
initial_values["L_s"] = df.loc[("Institution", "Household"), ("Factor", "Labor")]
initial_values["K_s"] = df.loc[("Institution", "Firms"), ("Factor", "Capital")]

initial_values["income"] = (
    initial_values["w"] * initial_values["L_s"] + initial_values["r"] * initial_values["K_s"]
)
initial_values["tau_income"] = initial_values["income_tax"] / initial_values["income"]
initial_values["net_income"] = (1 - initial_values["tau_income"]) * initial_values["income"]

for sector in sectors:
    long_sector = short_to_long[sector]
    initial_values[f"L_d_{sector}"] = (
        df.loc[("Factor", "Labor"), ("Activities", long_sector)] / initial_values["w"]
    )
    initial_values[f"K_d_{sector}"] = (
        df.loc[("Factor", "Capital"), ("Activities", long_sector)] / initial_values["r"]
    )
    initial_values[f"C_G_{sector}"] = df.loc[("Production", long_sector), ("Institution", "Govt")]

    import_tax = initial_values[f"import_tax_{sector}"]
    initial_values[f"sales_tax_{sector}"] = (
        df.loc[("Institution", "Govt"), ("Production", long_sector)] - import_tax
    )
    initial_values[f"C_{sector}"] = df.loc[
        ("Production", long_sector), ("Institution", "Household")
    ]

    # Excess demand -- What is this in terms of imports/exports?
    world_supply = df.loc[("Other", "Rest of World"), ("Production", long_sector)]
    world_demand = df.loc[("Production", long_sector), ("Other", "Rest of World")]
    initial_values[f"E_d_{sector}"] = (world_supply - world_demand) + import_tax

    initial_values[f"I_d_{sector}"] = df.loc[
        ("Production", long_sector), ("Other", "Capital Accumulation")
    ]

    # This computes Y, but what does this ultimately work out to? There is a lot of cancellations happening.
    total_i = (
        df.sum(axis=0).loc[("Production", long_sector)]
        - df.loc[("Production", long_sector), ("Other", "Rest of World")]
    )
    initial_values[f"Y_{sector}"] = total_i - initial_values[f"E_d_{sector}"]

    # Work out the tax rate from tax reciepts

    T_sec = initial_values[f"sales_tax_{sector}"]
    Y_sec = initial_values[f"Y_{sector}"]
    P_sec = initial_values[f"P_{sector}"]
    Ed_sec = initial_values[f"E_d_{sector}"]

    initial_values[f"tau_{sector}"] = T_sec / (Y_sec - T_sec)
    initial_values[f"P_Y_{sector}"] = P_sec / (1 + initial_values[f"tau_{sector}"])
    initial_values[f"tau_m_{sector}"] = (
        import_tax / (Ed_sec * P_sec - import_tax) if Ed_sec > 0 else 0
    )
    initial_values[f"P_w_{sector}"] = P_sec / (1 + initial_values[f"tau_m_{sector}"])

# Adjust values in SAM to account for non-normalized prices
for sector in sectors:
    long_sector = short_to_long[sector]
    for sector_j in sectors:
        long_sector_j = short_to_long[sector_j]
        SAM_idx = ("Production", long_sector), ("Activities", long_sector_j)
        initial_values[f"X_{sector}_{sector_j}"] = df.loc[SAM_idx]

inital_values_temp = initial_values.copy()
for key in initial_values:
    if key not in mod.unpacked_parameter_names + mod.unpacked_variable_names:
        del inital_values_temp[key]
initial_values = inital_values_temp

In [56]:
init_res = mod.calibrate(initial_values)
init_res.success

True

In [57]:
pd.set_option("display.max_rows", 100)

In [58]:
init_res.to_frame().head(100)

,initial,fitted
Y_Ag,5650.000000,5650.000000
Y_Ind,10820.000000,10820.000000
Y_Serv,10480.000000,10480.000000
VA_Ag,NaN,1350.000000
VA_Ind,NaN,3500.000000
VA_Serv,NaN,4700.000000
VC_Ag,NaN,4150.000000
VC_Ind,NaN,6300.000000
VC_Serv,NaN,5500.000000
X_Ag_Ag,1100.000000,1100.000000


In [59]:
init_res.to_frame().loc[lambda x: [idx.startswith("tau_m") for idx in x.index]]

,initial,fitted
tau_m_Ag,0.000000,0.000000
tau_m_Ind,0.697674,0.697674
tau_m_Serv,0.000000,0.000000


In [61]:
tax_cut = mod.simulate(
    init_res,
    final_delta_pct={"tau_m_Ind": 0.5},
    n_iter_euler=0,
    name="50% tariff tax cut",
    method="trust-krylov",
)

In [65]:
tax_cut.to_frame().head(60)

,initial,fitted
Y_Ag,5650.000000,5650.000379
Y_Ind,10820.000000,10819.997441
Y_Serv,10480.000000,10480.001639
VA_Ag,1350.000000,1350.000091
VA_Ind,3500.000000,3499.999173
VA_Serv,4700.000000,4700.000735
VC_Ag,4150.000000,4150.000277
VC_Ind,6300.000000,6299.998509
VC_Serv,5500.000000,5500.000859
X_Ag_Ag,1100.000000,1100.000073
